In [ ]:
%pip -q install google-genai
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [ ]:
# Configura a API Key do Google Gemini

import os
import io
import glob
from IPython.display import display, Markdown
import matplotlib.pyplot as plt

from google.colab import userdata
from google.colab import files

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai
from google.genai import types

client = genai.Client()

MODEL_ID = "gemini-2.5-flash-preview-04-17"

In [ ]:
# Imports para os agents

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from datetime import date
import textwrap # Para formatar melhor a saída de texto
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

ImportError: cannot import name 'google_docs' from 'google.adk.tools' (/usr/local/lib/python3.11/dist-packages/google/adk/tools/__init__.py)

In [ ]:
#for model in client.models.list():
#    print(model.name)

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# prompt: solicitar o upload de arquivos pdf das faturas.

# Solicitar upload de arquivos PDF
uploaded = files.upload()

# Processar os arquivos PDF uploaded
for filename, content in uploaded.items():
    print(f'Arquivo "{filename}" uploaded com sucesso.')
    # Aqui você pode adicionar a lógica para analisar o conteúdo do PDF
    # Por exemplo, se você tiver uma função `analyze_pdf_with_gemini` que recebe o conteúdo do PDF e retorna a análise
    # analysis_result = analyze_pdf_with_gemini(content)
    # display(Markdown(f'Análise do arquivo "{filename}":\n\n{analysis_result}'))

    # Exemplo simples: apenas imprimir o tamanho do arquivo
    display(Markdown(f'Tamanho do arquivo "{filename}": {len(content)} bytes'))

    # Se você quiser ler o conteúdo como texto (requer bibliotecas como PyMuPDF ou pdfminer.six)
    # !pip install pymupdf
    # import fitz # PyMuPDF
    # try:
    #     pdf_document = fitz.open(stream=io.BytesIO(content), filetype="pdf")
    #     text = ""
    #     for page_num in range(pdf_document.page_count):
    #         page = pdf_document.load_page(page_num)
    #         text += page.get_text()
    #     display(Markdown(f'Conteúdo de texto do arquivo "{filename}":\n\n{text[:500]}...')) # Exibindo apenas os primeiros 500 caracteres
    # except Exception as e:
    #     print(f"Erro ao ler o conteúdo do PDF {filename}: {e}")

    # Se você quiser enviar o conteúdo binário para o Gemini para análise (dependendo do modelo e da API)
    # Note: A API do Gemini pode ter limitações no tamanho e formato dos arquivos.
    # content_part = genai.types.content_types.to_content({"mime_type": "application/pdf", "data": content})
    # model = client.models.get(MODEL_ID) # Certifique-se de que o MODEL_ID suporta processamento de PDF
    # try:
    #     # Este é um exemplo genérico, a forma exata de passar o arquivo pode variar
    #     # Verifique a documentação mais recente da API do Gemini para processamento de arquivos
    #     response = model.generate_content([content_part, "Analise este arquivo PDF e resuma as informações principais."])
    #     display(Markdown(f'Resposta do Gemini para "{filename}":\n\n{response.text}'))
    # except Exception as e:
    #     print(f"Erro ao enviar o arquivo {filename} para o Gemini: {e}")

# Lembre-se de que a capacidade de analisar diretamente arquivos PDF binários depende do modelo Gemini específico e da API.
# Para análises mais complexas de PDF, você pode precisar de bibliotecas Python para extrair texto e imagens antes de enviar para o Gemini.

Saving fatura 202307.pdf to fatura 202307.pdf
Saving OUROCARD_PLATINUM_ESTILO_VISA-Jul_23.pdf to OUROCARD_PLATINUM_ESTILO_VISA-Jul_23.pdf
Arquivo "fatura 202307.pdf" uploaded com sucesso.


Tamanho do arquivo "fatura 202307.pdf": 384874 bytes

Arquivo "OUROCARD_PLATINUM_ESTILO_VISA-Jul_23.pdf" uploaded com sucesso.


Tamanho do arquivo "OUROCARD_PLATINUM_ESTILO_VISA-Jul_23.pdf": 359869 bytes

In [ ]:
pdf_files = glob.glob('*.pdf')
prompt_parts = list()

if not pdf_files:
    print("Nenhum arquivo PDF encontrado na pasta atual.")
else:
    print("Arquivos PDF encontrados:")
    for pdf_file in pdf_files:
        print(f"- {pdf_file}")
        try:
            with open(pdf_file, "rb") as f:
                binary_content = f.read()
            print(f"Arquivo '{pdf_file}' carregado com sucesso. Tamanho: {len(binary_content)} bytes")
            # Agora 'binary_content' contém os dados binários do arquivo
            # Você pode processá-lo conforme necessário
            # Prepare the PDF content part directly as a dictionary
            pdf_part_data = types.Part.from_bytes(mime_type="application/pdf", data = binary_content)

            # Combine parts into a list for the prompt
            prompt_parts.append(pdf_part_data)
        except FileNotFoundError:
            print(f"Erro: Arquivo '{pdf_file}' não encontrado na pasta atual.")
        except Exception as e:
            print(f"Ocorreu um erro ao carregar o arquivo: {e}")
    #for filename, file_content in uploaded.items():


    # Optionally, add a text part (e.g., instructions)
    text_part = {"text": "Analise as faturas de cartões de crédito destes arquivos pdf e separe os lançamentos em um formado de planilha. Junte os lançamentos de todas as faturas em uma única listagem."}

    prompt_parts.append(text_part)
    try:
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt_parts,
        )
        display(Markdown(f'Resposta do Gemini ":\n\n{response.text}'))
        Lancamentos = response.text
    except Exception as e:
        print(f"Erro ao enviar os arquivos para o Gemini: {e}")


Arquivos PDF encontrados:
- fatura 202307.pdf
Arquivo 'fatura 202307.pdf' carregado com sucesso. Tamanho: 384874 bytes
- OUROCARD_PLATINUM_ESTILO_VISA-Jul_23.pdf
Arquivo 'OUROCARD_PLATINUM_ESTILO_VISA-Jul_23.pdf' carregado com sucesso. Tamanho: 359869 bytes


Resposta do Gemini ":

Okay, here are the transactions extracted from both the Caixa and Banco do Brasil credit card statements, combined into a single list in a spreadsheet-like format.

| Card Issuer       | Date       | Description                      | Installment | Amount    |
| :---------------- | :--------- | :------------------------------- | :---------- | :-------- |
| Caixa             | 04/03      | DROGARIAS PACHECO                | 4/6         | 38.52     |
| Caixa             | 06/04      | SUPERMERCADOS BH                 | 3/3         | 150.45    |
| Caixa             | 13/05      | SELLOS ACESSORIOS                | 2/2         | 64.00     |
| Caixa             | 20/05      | PICCADILLY C*                    | 2/10        | 38.69     |
| BB                | 20/05      | PICCADILLY C*                    | 2/10        | 38.69     |
| Caixa             | 31/05      | AMAZON BR                        | 2/6         | 57.44     |
| BB                | 31/05      | AMAZON BR                        | 2/6         | 57.44     |
| Caixa             | 05/06      | SUPERMERCADOS BH                 |             | 76.29     |
| Caixa             | 05/06      | FEM ALIMENTOS                    |             | 13.00     |
| BB                | 05/06      | Amazon.com.br                    |             | 40.66     |
| BB                | 05/07      | LOCALIZA RAC ACBHZO              |             | 50.29     |
| Caixa             | 07/06      | SUPERMERCADOS BH                 | 1/2         | 88.83     |
| Caixa             | 08/06      | PADARIA PAO EXPRESSO             |             | 25.95     |
| Caixa             | 08/06      | MP *REIDOMARMITEX                |             | 20.00     |
| Caixa             | 08/06      | FISIA NIKE ECOMMER               | 1/2         | 100.92    |
| BB                | 08/06      | FISIA NIKE ECOMMER               | 1/2         | 100.92    |
| BB                | 08/10      | ZINZANE                          | 8/10        | 113.99    |
| Caixa             | 09/05      | Magalu *MAGAZ                    | 3/8         | 66.11     |
| BB                | 09/05      | Magalu *MAGAZ                    | 3/8         | 66.11     |
| Caixa             | 09/06      | SUPERMERCADOS BH                 |             | 22.98     |
| Caixa             | 09/06      | SUPERMERCADOS BH                 |             | 115.02    |
| Caixa             | 10/06      | Drogarias Pacheco S.A. (Cartão 7632) |             | 38.46     |
| BB                | 10/06      | AMAZON MARKET                    | 2/2         | 71.91     |
| Caixa             | 10/06      | SUPERMERCADOS BH                 |             | 148.83    |
| BB                | 10/11      | MERCADOLIVRE*                    | 10/10       | 66.21     |
| Caixa             | 10/11      | ZINZANE                          | 8/10        | 113.99    |
| BB                | 13/04      | MercPag*SAMAR                    | 3/4         | 34.54     |
| Caixa             | 13/04      | HNA *O Bot                       | 3/9         | 21.20     |
| BB                | 13/04      | HNA *O Bot                       | 3/9         | 21.20     |
| BB                | 13/11      | MERCADOLIVRE*                    | 8/10        | 21.17     |
| Caixa             | 11/06      | DROGARIAS PACHECO                |             | 43.45     |
| Caixa             | 11/06      | SUPERMERCADOS BH                 |             | 33.36     |
| Caixa             | 11/06      | UNIFIP *MENSALIDADE              |             | 1224.47   |
| Caixa             | 12/06      | Uber *UBER *TRIP                 |             | 12.90     |
| Caixa             | 12/06      | Uber *UBER *TRIP                 |             | 8.96      |
| Caixa             | 12/06      | Uber *UBER *TRIP                 |             | 6.98      |
| Caixa             | 12/06      | SUPERMERCADOS BH                 |             | 61.58     |
| Caixa             | 12/06      | TOTAL FIT MONTES CLARO           |             | 108.14    |
| BB                | 14/10      | Nestle Brasil                    | 9/10        | 8.36      |
| BB                | 14/06      | Nestle Brasil                    | 1/2         | 11.13     |
| Caixa             | 14/06      | DROGARIAS PACHECO                | 1/2         | 58.07     |
| Caixa             | 14/06      | Uber *UBER *TRIP                 |             | 10.92     |
| Caixa             | 14/06      | Uber *UBER *TRIP                 |             | 9.94      |
| Caixa             | 14/06      | Uber *UBER *TRIP                 |             | 10.96     |
| Caixa             | 14/06      | SUPERMERCADOS BH                 |             | 91.54     |
| Caixa             | 14/06      | SUPERMERCADOS BH                 |             | 65.92     |
| BB                | 15/06      | NETFLIX.COM                      |             | 25.90     |
| BB                | 15/06      | GERALDO DE PAULA VALLE           |             | 84.70     |
| Caixa             | 15/06      | Uber *UBER *TRIP                 |             | 10.98     |
| Caixa             | 15/06      | Uber *UBER *TRIP                 |             | 9.92      |
| Caixa             | 15/06      | Uber *UBER *TRIP                 |             | 10.91     |
| Caixa             | 15/06      | Uber *UBER *TRIP                 |             | 9.96      |
| Caixa             | 15/06      | Uber *UBER *TRIP                 |             | 8.96      |
| Caixa             | 15/06      | Uber *UBER *TRIP                 |             | 8.90      |
| Caixa             | 15/06      | PONTAL TECIDOS MOC               |             | 50.94     |
| BB                | 15/12      | Nestle Brasil                    | 7/10        | 14.00     |
| Caixa             | 16/05      | Nestle Brasil                    | 2/10        | 11.13     |
| BB                | 16/05      | Nestle Brasil                    | 2/10        | 11.13     |
| BB                | 16/06      | EC *MERCADOLIVRE                 |             | 149.00    |
| BB                | 16/06      | FARMACIA REAL LTDA               |             | 31.98     |
| Caixa             | 16/06      | Uber *UBER *TRIP                 |             | 14.92     |
| Caixa             | 16/06      | Uber *UBER *TRIP                 |             | 10.97     |
| Caixa             | 16/06      | Uber *UBER *TRIP                 |             | 9.93      |
| Caixa             | 16/06      | Uber *UBER *TRIP                 |             | 9.91      |
| Caixa             | 16/06      | SUPERMERCADOS BH                 |             | 93.82     |
| Caixa             | 16/06      | CENTRO DE ESTETICA FAS           |             | 77.00     |
| Caixa             | 16/06      | FARMACIA REAL LTDA               |             | 31.98     |
| Caixa             | 16/06      | CASA DO BOLO IMACULADA           |             | 26.00     |
| Caixa             | 16/06      | CASA DOS AVIAMENTOS              |             | 10.40     |
| BB                | 17/06      | EC *OKAYSHOP                     |             | 109.71    |
| BB                | 17/06      | PGO*BENDITO MALTE                |             | 88.00     |
| BB                | 17/06      | PGO*BENDITO MALTE                |             | 60.00     |
| BB                | 17/06      | KARSTEN                          | 1/3         | 105.41    |
| Caixa             | 17/06      | SUPERMERCADOS BH                 |             | 62.89     |
| Caixa             | 17/06      | REI DO MARMITEX MOC              |             | 14.00     |
| BB                | 17/09      | MERCADOLIVRE*                    | 10/10       | 66.21     |
| BB                | 18/05      | SUPERMERCADOS PARC               | 2/3         | 78.39     |
| Caixa             | 18/05      | SUPERMERCADOS PARC               | 2/3         | 78.39     |
| BB                | 18/06      | EC *NIVEL6                       |             | 14.99     |
| BB                | 18/06      | PADARIA PAO EXPRESSO             |             | 11.86     |
| BB                | 18/06      | SUPERMERCADOS BH                 |             | 11.29     |
| BB                | 18/06      | CASA DA ESFIHA                   |             | 52.85     |
| Caixa             | 18/06      | SUPERMERCADOS BH                 |             | 108.24    |
| BB                | 19/11      | MLP *MAGAZ                       | 8/10        | 51.19     |
| BB                | 19/06      | PG *DIAL VIAG PED                |             | 69.90     |
| Caixa             | 19/06      | Uber *UBER *TRIP                 |             | 11.93     |
| Caixa             | 19/06      | Uber *UBER *TRIP                 |             | 10.91     |
| Caixa             | 19/06      | SUPERMERCADOS BH                 |             | 89.55     |
| Caixa             | 19/06      | PG *DIAL VIAG PED                |             | 69.90     |
| Caixa             | 19/06      | Google Globo                     |             | 19.90     |
| Caixa             | 19/06      | Uber *UBER *TRIP                 |             | 9.98      |
| Caixa             | 19/06      | Uber *UBER *TRIP                 |             | 9.98      |
| Caixa             | 19/06      | Uber *UBER *TRIP                 |             | 9.96      |
| BB                | 20/02      | Nestle Brasil                    | 5/10        | 10.74     |
| BB                | 20/12      | GRUPO DLT                        | 7/12        | 185.00    |
| Caixa             | 20/06      | Uber *UBER *TRIP                 |             | 9.98      |
| BB                | 21/01      | AMAZON BR                        | 6/7         | 50.98     |
| Caixa             | 21/06      | SUPERMERCADOS BH                 |             | 11.82     |
| Caixa             | 21/06      | Uber *UBER *TRIP                 |             | 9.97      |
| BB                | 21/06      | SUPERMERCADOS PARC               | 1/3         | 85.12     |
| BB                | 22/06      | SUPERMERCADOS BH                 |             | 26.90     |
| BB                | 22/06      | SUPERMERCADOS BH                 |             | 19.68     |
| BB                | 22/06      | DROGARIAS PACHECO                |             | 14.09     |
| Caixa             | 22/06      | SUPERMERCADOS BH                 |             | 83.67     |
| Caixa             | 22/06      | Uber *UBER *TRIP                 |             | 11.92     |
| Caixa             | 22/06      | Uber *UBER *TRIP                 |             | 9.97      |
| Caixa             | 22/06      | Uber *UBER *TRIP                 |             | 9.91      |
| BB                | 02/06      | LAPIEL                           | 2/3         | 59.33     |
| BB                | 02/06      | PAG*DROGMINASBRASIL              | 2/2         | 51.40     |
| Caixa             | 02/07      | SUPERMERCADOS BH                 |             | 98.35     |
| BB                | 03/06      | PIZARRO JEANS                    | 2/2         | 53.85     |
| BB                | 03/06      | CENTRO DE ESTETICA FAS           | 2/4         | 158.50    |
| BB                | 03/07      | AMAZON BR                        | 1/4         | 50.69     |
| BB                | 03/07      | AMAZON BR                        | 1/3         | 61.71     |
| Caixa             | 03/07      | TOTAL FIT MONTES CLARO           |             | 99.95     |
| BB                | 23/06      | SUPERMERCADOS BH                 |             | 49.80     |
| BB                | 23/06      | CENTRO DE ESTETICA FAS           |             | 37.00     |
| BB                | 23/06      | DOLAR SHOPPING                   |             | 18.40     |
| BB                | 23/06      | Uber *UBER *TRIP                 |             | 9.92      |
| BB                | 23/06      | Uber *UBER *TRIP                 |             | 8.97      |
| BB                | 23/06      | Uber *UBER *TRIP                 |             | 7.97      |
| BB                | 23/06      | Uber *UBER *TRIP                 |             | 7.95      |
| Caixa             | 23/06      | SUPERMERCADOS BH                 |             | 49.80     |
| Caixa             | 23/06      | CENTRO DE ESTETICA FAS           |             | 37.00     |
| Caixa             | 23/06      | DOLAR SHOPPING                   |             | 18.40     |
| Caixa             | 23/06      | Uber *UBER *TRIP                 |             | 9.92      |
| Caixa             | 23/06      | Uber *UBER *TRIP                 |             | 8.97      |
| Caixa             | 23/06      | Uber *UBER *TRIP                 |             | 7.97      |
| Caixa             | 23/06      | Uber *UBER *TRIP                 |             | 7.95      |
| BB                | 24/06      | G G FERRAGENS COLONIAI           |             | 40.00     |
| BB                | 24/06      | REINO ANIMAL                     |             | 7.80      |
| BB                | 24/06      | SUPERMERCADOS BH                 |             | 87.77     |
| BB                | 24/06      | SR. PET CARE                     |             | 45.00     |
| BB                | 24/06      | CASA DA ESFIHA                   |             | 52.85     |
| BB                | 24/11      | KALUN                            | 8/10        | 15.90     |
| BB                | 24/06      | PADARIA PAO EXPRESSO             |             | 8.26      |
| BB                | 24/06      | PEIXARIA DO TONY                 |             | 163.00    |
| BB                | 25/02      | PAGUE MENOS 6                    | 5/6         | 50.58     |
| BB                | 25/02      | PAGUE MENOS 6                    | 5/6         | 60.90     |
| BB                | 25/06      | GIRAFFAS                         |             | 115.00    |
| BB                | 25/06      | PADARIA PAO EXPRESSO             |             | 8.26      |
| BB                | 25/06      | CAN TINA DA CASA DELIVE          |             | 62.00     |
| BB                | 25/06      | SUPERMERCADOS BH                 |             | 11.63     |
| BB                | 25/06      | SUPERMERCADOS BH                 |             | 49.31     |
| Caixa             | 25/06      | GIRAFFAS                         |             | 115.00    |
| Caixa             | 26/04      | SUPERMERCADOS PARC               | 3/3         | 57.99     |
| BB                | 26/04      | SUPERMERCADOS PARC               | 3/3         | 57.99     |
| BB                | 26/09      | TELECIMENTO A                    | 10/10       | 104.95    |
| BB                | 26/06      | SUPERMERCADOS BH                 |             | 89.46     |
| BB                | 26/06      | CAN TINA DA CASA DELIVE          |             | 62.00     |
| Caixa             | 26/06      | Uber *UBER *TRIP                 |             | 9.96      |
| Caixa             | 26/06      | Uber *UBER *TRIP                 |             | 9.93      |
| Caixa             | 26/06      | Uber *UBER *TRIP                 |             | 9.92      |
| BB                | 27/04      | PAG*Chaparral                    | 3/3         | 258.17    |
| BB                | 27/06      | SUPERMERCADOS PARC               | 1/3         | 44.10     |
| Caixa             | 27/06      | Uber *UBER *TRIP                 |             | 9.92      |
| Caixa             | 27/06      | Uber *UBER *TRIP                 |             | 8.90      |
| BB                | 28/04      | VENUS                            | 3/6         | 49.80     |
| BB                | 28/06      | SUPERMERCADOS PARC               | 1/3         | 51.54     |
| Caixa             | 28/06      | Uber *UBER *TRIP                 |             | 8.91      |
| BB                | 29/06      | Uber *UBER *TRIP                 |             | 9.99      |
| BB                | 29/06      | SUPERMERCADOS BH                 |             | 56.32      |
| BB                | 30/03      | MLP *MAGAZ                       | 4/5         | 46.45     |
| BB                | 30/11      | AMAZON BR                        | 8/9         | 55.55     |
| BB                | 30/06      | FEM ALIMENTOS                    |             | 17.72     |
| BB                | 30/06      | DROGARIAS PACHECO                |             | 63.87     |
| BB                | 30/06      | SUPERMERCADOS BH                 |             | 52.99     |
| Caixa             | 30/06      | FEM ALIMENTOS                    |             | 17.72     |
| Caixa             |            | ANUIDADE DIFERENCIADA ADI        |             | 8.62      |
| Caixa             |            | ANUIDADE                         |             | 17.25     |
| Caixa             |            | ANUIDADE DIFERENCIADA ADI        |             | 8.62      |

**Notes:**
*   Dates are listed in DD/MM format as they appear in the statements.
*   Amounts are in BRL (Brazilian Reais).
*   Installment information is extracted where available for parcelled purchases.
*   Summary lines (like balance forward, total payments, total per category) are excluded.
*   Caixa statements sometimes listed the same ANUIDADE amount multiple times; both instances are included.
*   One transaction (PICCADILLY C*) appeared on both statements with the same date and amount; this could be a duplicate entry or a transaction involving both accounts somehow, it is listed as it appears.

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
###################################################
# --- Agente 1: Caracterizador de Lançamentos --- #
###################################################
def agente_categorizador(lancamentos):

    categorizador = Agent(
        name="agente_categorizador",
        model=MODEL_ID,
        instruction="""
        Você é um categorizador de lançamentos de cartões de crédito.
        Retorne um resumo como a soma dos lançamentos por categoria.
        """,
        description="Agente que categoriza os lançamentos das faturas de cartões de crédito",

        # tools=[google_search]
    )

    entrada_do_agente_categorizador = f"Lançamentos: {lancamentos}"

    categorias = call_agent(categorizador, entrada_do_agente_categorizador)
    return categorias


In [ ]:
##################################################################################################
# --- Agente 2: Cria um gráfico --- #
##################################################################################################
def agente_grafico(categorias):
    grafico = Agent(
        name="grafico",
        model=MODEL_ID,
        # Inserir as instruções do gerador_planilha #################################################
        instruction="""
        Com base nos seguintes dados de categorias e suas somas, gere um código Python (utilizando bibliotecas como matplotlib ou plotly) para criar um gráfico de pizza.
        """,
        description="Agente que gerar gráfico",
        #tools=[google_docs]
    )

    entrada_do_agente_grafico = f"Categorias: {categorias}"
    # Executa o agente
    Grafico = call_agent(grafico, entrada_do_agente_grafico)
    return Grafico

In [ ]:
print("🚀 Iniciando o Sistema Análise de Faturas de Cartões de Crédito com 2 Agentes 🚀")

# --- Solicitar o upload dos pdfs das faturas ---
#topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
#if not topico:
#    print("Você esqueceu de digitar o tópico!")
#else:
#    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

categorias = agente_categorizador(Lancamentos)
print("\n--- 📝 Resultado do Agente 1 (Categorizador) ---\n")
display(to_markdown(categorias))
print("--------------------------------------------------------------")

#Grafico = agente_grafico(categorias)
#print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
#display(Grafico)
#print("--------------------------------------------------------------")



🚀 Iniciando o Sistema Análise de Faturas de Cartões de Crédito com 2 Agentes 🚀

--- 📝 Resultado do Agente 1 (Categorizador) ---



> Ok, aqui está o resumo dos seus lançamentos por categoria, somando os valores de ambas as faturas:
> 
> **Resumo de Lançamentos por Categoria**
> 
> *   **Alimentação/Mercado:** R$ 3014,99
> *   **Educação:** R$ 1224,47
> *   **Compras Online:** R$ 1220,27
> *   **Outros Serviços/Compras Diversas:** R$ 855,32
> *   **Vestuário/Acessórios:** R$ 620,82
> *   **Saúde/Bem-Estar:** R$ 517,59
> *   **Farmácia/Saúde:** R$ 483,30
> *   **Transporte:** R$ 411,62
> *   **Viagem:** R$ 139,80
> *   **Pet Shop:** R$ 52,80
> *   **Assinaturas Digitais:** R$ 45,80
> *   **Anuidade:** R$ 34,49
> 
> **Total Geral:** R$ 8621,43


--------------------------------------------------------------


AttributeError: 'function' object has no attribute 'name'